In [3]:
s =  'This is a book \nNo'
s.strip()

'This is a book \nNo'

In [6]:
s = 's==ss==\n'
s.split('==')

['s', 'ss', '\n']

In [8]:
from transformers import BertModel, BertTokenizer, BertConfig

config = BertConfig.from_pretrained('bert-base-chinese', output_hidden_states=True)
bert = BertModel.from_pretrained('bert-base-chinese', config=config)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
